In [2]:
import numpy as np
import pandas as pd
import os

# 1. Table Prep
Prepare the table: add names, make the format readable, etc.

In [3]:
# read in the distinctiveness ratings
dist_rat_df = pd.read_csv("Average_rock_distinct_ratings.txt", 
                          sep=r"\s+", header=None, 
                          names=['image_id', 'Category','Item', 'Distinct_Rating'])
dist_rat_df.head()

,image_id,Category,Item,Distinct_Rating
0,1,1,1,3.99
1,2,1,2,3.93
2,3,1,3,3.17
3,4,1,4,3.18
4,5,1,5,3.32


In [4]:
# Extract filenames and add them to the DataFrame
filenames = sorted(os.listdir("Stimuli_Rocks480"))
dist_rat_df["image_png"] = filenames
dist_rat_df["Broad_Category_Name"] = dist_rat_df["image_png"].str.split("_").str[0]
dist_rat_df["Category_Name"] = dist_rat_df["image_png"].str.split("_").str[1]

dist_rat_df.head()

,image_id,Category,Item,Distinct_Rating,image_png,Broad_Category_Name,Category_Name
0,1,1,1,3.99,I_Andesite_01.png,I,Andesite
1,2,1,2,3.93,I_Andesite_02.png,I,Andesite
2,3,1,3,3.17,I_Andesite_03.png,I,Andesite
3,4,1,4,3.18,I_Andesite_04.png,I,Andesite
4,5,1,5,3.32,I_Andesite_05.png,I,Andesite


# 2. Group Creation
Create two groups based on their distinctiveness ratings

In [5]:
# Sort by Category first to keep order stable
dist_rat_df = dist_rat_df.sort_values(["Category", "Distinct_Rating"], ascending=[True, False]).copy()

# We'll store the results here
dist_rat_df["Group1"] = ""
dist_rat_df["Group2"] = ""

In [6]:
# Process each category separately
start_with_target = True  # Category 1 starts with Target in Group1
for cat, group_df in dist_rat_df.groupby("Category", sort=False):
    # Determine the pattern for this category
    if start_with_target:
        patterns = [("Target", "Foil"), ("Foil", "Target")]
    else:
        patterns = [("Foil", "Target"), ("Target", "Foil")]
    
    # Assign patterns row by row
    for i, idx in enumerate(group_df.index):
        g1, g2 = patterns[i % 2]  # alternate
        dist_rat_df.at[idx, "Group1"] = g1
        dist_rat_df.at[idx, "Group2"] = g2
    
    # Flip pattern for next category
    start_with_target = not start_with_target

# Reset index
dist_rat_df = dist_rat_df.reset_index(drop=True)

In [7]:
# Make sure no objects are both targets and foils
(dist_rat_df["Group1"] == dist_rat_df["Group2"]).sum()

0

In [8]:
# Reorder columns
dist_rat_df = dist_rat_df[['image_id', 'Broad_Category_Name', 'Category', 'Category_Name', 'Item', 'Distinct_Rating', 'Group1','Group2','image_png']]
dist_rat_df.head()

,image_id,Broad_Category_Name,Category,Category_Name,Item,Distinct_Rating,Group1,Group2,image_png
0,7,I,1,Andesite,7,4.81,Target,Foil,I_Andesite_07.png
1,14,I,1,Andesite,14,4.47,Foil,Target,I_Andesite_14.png
2,1,I,1,Andesite,1,3.99,Target,Foil,I_Andesite_01.png
3,15,I,1,Andesite,15,3.98,Foil,Target,I_Andesite_15.png
4,2,I,1,Andesite,2,3.93,Target,Foil,I_Andesite_02.png


Columns:

1. `image_id`: Order number in the original list (and in the folder)
2. `Broad_Category_Name`: I - igneous, M - metamorphic, S - sedimentary rocks
3. `Category`: Category ID
4. `Category_Name`
5. `Item`: There are 16 items of each category
6. `Distinct_Rating`: Average distinctiveness ratings from Meagher & Nosofsky (2023)
7. `Group1`: Whether this object is a target (old) or a foil (new) for the first group of subjects (opposite to `Group2`)
8. `Group2`: Whether this object is a target (old) or a foil (new) for the second group of subjects (opposite to `Group1`)
9. `image_png`: Image file name with this object

In [10]:
# Save data
dist_rat_df.to_csv("Average_Distinct_Ratings_480Rocks.csv", index=False)
dist_rat_df.to_json("Average_Distinct_Ratings_480Rocks.json", orient="records", indent=2)

In [9]:
dist_rat_df

,image_id,Broad_Category_Name,Category,Category_Name,Item,Distinct_Rating,Group1,Group2,image_png
0,7,I,1,Andesite,7,4.81,Target,Foil,I_Andesite_07.png
1,14,I,1,Andesite,14,4.47,Foil,Target,I_Andesite_14.png
2,1,I,1,Andesite,1,3.99,Target,Foil,I_Andesite_01.png
3,15,I,1,Andesite,15,3.98,Foil,Target,I_Andesite_15.png
4,2,I,1,Andesite,2,3.93,Target,Foil,I_Andesite_02.png
...,...,...,...,...,...,...,...,...,...
475,468,S,30,Shale,4,2.23,Target,Foil,S_Shale_04.png
476,467,S,30,Shale,3,1.95,Foil,Target,S_Shale_03.png
477,474,S,30,Shale,10,1.91,Target,Foil,S_Shale_10.png
478,477,S,30,Shale,13,1.88,Foil,Target,S_Shale_13.png
